# Binary classifier to predict NV-HAP versus VAP (presence absence data)

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from xgboost import XGBClassifier
import shap
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, average_precision_score

cwd = Path.cwd()
print(cwd)
datasets = cwd / '../results/tax_classification_out/abundance_matrices'
results = cwd / '../results/ML_out'

/mnt/c/git_repos/pneumonia/ML_scripts


/home/cednotsed/miniconda3/envs/mashin/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Preprocessing

### Load data

In [2]:
raw_df = pd.read_csv(datasets / 'RA.G.zeroed.decontam.2.csv')
meta = pd.read_csv(cwd / "../data/metadata/parsed_patient_metadata.filt.csv")
# display(raw_df)
# display(meta)

merged_df = raw_df.merge(meta, on='run_id', how='left')
merged_filt = merged_df.loc[merged_df.hap_vap2.isin(['NV-HAP', 'VAP']), :]

# Response
y = merged_filt.loc[:, 'hap_vap2'].copy()

# Features
X = merged_filt.loc[:, ~merged_filt.keys().isin(meta.keys())].copy()
X[X > 0] = 1

# # Add sample type
# X.insert(0, 'sample_type', merged_filt.loc[:, 'sample_type'])

# # Categorical encode variables
# X.replace({'sample_type': {'SPU' : 1, 'ETT' : 2, 'ND-BAL' : 3, 'BAL': 4}}, inplace=True)

# Rename features
X.columns = X.columns.str.replace('[^A-Za-z0-9]+', '_') 
print(X.shape)
print(y.shape)

# Binary encode y
y.loc[y == 'NV-HAP'] = 1
y.loc[y == 'VAP'] = 0
y = y.astype('int')
y.value_counts()

(171, 152)
(171,)


hap_vap2
0    127
1     44
Name: count, dtype: int64

In [3]:
n_splits = 5

pos = len(y[y == 1])
neg = len(y[y == 0])
split_sizes = pd.DataFrame({'NV-HAP': [pos - int(pos / n_splits), int(pos / n_splits)], 
                           'VAP': [neg - int(neg / n_splits), int(neg / n_splits)]}, index=['Train fold', 'Test fold'])

display(split_sizes)

# Get negative to positive ratio
ratio = sum(y == 0) / sum(y == 1)

,NV-HAP,VAP
Train fold,36,102
Test fold,8,25


In [4]:
def optimise_evaluate(X, y):
    np.random.seed(66)
    ratio = sum(y == 0) / sum(y == 1)
    
    # Hyperparemeter Optimisation using grid search (F1)
    model = XGBClassifier()
    n_estimators = range(100, 1000, 100)
    max_depth = range(1, 5, 1)
    # gamma = np.linspace(0.1, 3, 10)
    colsample_bytree = np.linspace(0.1, 1, 10)
    
    param_grid = dict(max_depth=max_depth, 
                      n_estimators=n_estimators, 
                      colsample_bytree=colsample_bytree)
                      # gamma=gamma,)
    
    inner_cv = StratifiedKFold(n_splits=n_splits, shuffle=True)
    outer_cv = StratifiedKFold(n_splits=n_splits, shuffle=True)

    # Inner CV
    model = GridSearchCV(model, 
                         param_grid, 
                         scoring='f1',
                         n_jobs=16, 
                         cv=inner_cv, 
                         verbose=1)

    model.fit(X, y)
    best_params = model.best_params_
    print(best_params)

    # Custom metrics
    precision = make_scorer(precision_score, average='binary')
    recall = make_scorer(recall_score, average='binary')
    f1 = make_scorer(f1_score, average='binary')
    auprc = make_scorer(average_precision_score, average=None)
    
    scoring = {'precision': precision, 
               'recall': recall, 
               'AUROC': 'roc_auc',
               'F1': f1}

    # Outer CV
    outer_results = cross_validate(model, X=X, y=y, cv=outer_cv, scoring=scoring)
    outer_results = pd.DataFrame(outer_results).mean()[['test_precision', 'test_recall', 'test_F1', 'test_AUROC']]

    return outer_results, best_params


In [5]:
raw_results, raw_params = optimise_evaluate(X, y)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
{'colsample_bytree': 0.1, 'max_depth': 2, 'n_estimators': 700}
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Fitting 5 folds for each of 360 candidates, totalling 1800 fits


In [6]:
raw_results

test_precision    0.510000
test_recall       0.338889
test_F1           0.389268
test_AUROC        0.653229
dtype: float64

In [7]:
print(raw_params)
res_df = pd.DataFrame(raw_results).transpose()
res_df.to_csv(results / 'raw_results/nvhap_vap.presence.csv', index=False)

{'colsample_bytree': 0.1, 'max_depth': 2, 'n_estimators': 700}
